# Fine-Tuning InternLM2 with LoRA on A100 GPU

## Overview

This notebook provides a detailed guide to fine-tuning the InternLM2 model for sequence classification tasks using Low-Rank Adaptation (LoRA) on an A100 GPU. The process covers everything from setting up the environment and installing dependencies to preprocessing data, configuring the model, and performing efficient training. The final steps include saving and exporting the fine-tuned model for deployment.

### Settings and Configuration

- **Model:** InternLM2 with Low-Rank Adaptation (LoRA)
- **Hardware:** GPU (A100, 40GB)
- **Precision:** Mixed Precision (FP16)
- **Batch Size:** 8 (per device)
- **Learning Rate:** 1e-6
- **Optimization Strategy:** AdamW with 8-bit precision
- **Number of Epochs:** 1
- **Training Data:** Custom dataset loaded from CSV
- **Tokenization:** Managed using AutoTokenizer with a max sequence length of 1024 tokens

### Key Features

- **Low-Rank Adaptation (LoRA):** Efficient adaptation of large models to specific tasks with reduced computational overhead.
- **GPU Acceleration:** Utilization of A100 GPUs for high-performance training.
- **Custom Data Processing:** Tailored preprocessing and tokenization to handle complex input data structures.
- **Efficient Model Saving:** Techniques for saving the trained model, including merging LoRA layers into the base model for seamless deployment.

### Created By

This notebook was created by Sathya Narayanan Balamurugan to demonstrate the efficient fine-tuning of large language models using LoRA on high-performance GPUs, with a focus on practical implementation and deployment.


# Login

In [ ]:
!pip install --upgrade huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="SathyaVrv/Lmsys-InternLM",local_dir='~/downloads')

# Install

In [1]:
!python3 -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.1 MB/s eta 0:00:00a 0:00:01
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of flatbuffers or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1


In [1]:
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 159.3 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 78.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 29.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 103.1 MB/s eta 0:00:0000:01
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to u

In [2]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 161.5 MB/s eta 0:00:0000:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.1
    Uninstalling tqdm-4.66.1:
      Successfully uninstalled tqdm-4.66.1
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1


In [3]:
!pip install einops

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.8 MB/s eta 0:00:00


In [5]:
pip install --upgrade pandas 

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 110.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install --upgrade jinja2

Defaulting to user installation because normal site-packages is not writeable


# Imports

In [1]:
import importlib.util
import sys
import os

# Specify the directory containing the modules
directory = 'modeling-internlm'

# Add the directory to sys.path so Python can find the modules
sys.path.append(directory)

# Load the configuration module first to avoid ImportError
config_spec = importlib.util.spec_from_file_location("configuration_internlm2", os.path.join(directory, "configuration_internlm2.py"))
configuration_internlm2 = importlib.util.module_from_spec(config_spec)
config_spec.loader.exec_module(configuration_internlm2)

# Now load the modeling_internlm2 module
spec = importlib.util.spec_from_file_location("modeling_internlm2", os.path.join(directory, "modeling_internlm2.py"))
modeling_internlm2 = importlib.util.module_from_spec(spec)
spec.loader.exec_module(modeling_internlm2)

# Add the modules to sys.modules
sys.modules["configuration_internlm2"] = configuration_internlm2
sys.modules["modeling_internlm2"] = modeling_internlm2

# Now you can import InternLM2Model
from modeling_internlm2 import *
from configuration_internlm2 import *

In [2]:
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from transformers import AutoModel, AutoConfig, AutoTokenizer
from sklearn.model_selection import train_test_split
import torch.nn as nn
import os
import time
import copy
from dataclasses import dataclass
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from scipy.special import softmax
from sklearn.preprocessing import LabelEncoder
from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType,PeftModel
from sklearn.metrics import log_loss, accuracy_score
from sklearn.utils import resample
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
from transformers.data.data_collator import pad_without_fast_tokenizer_warning

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ubuntu/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ubuntu/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [3]:
from sklearn.utils import resample
TRAIN_CSV = "data/train.csv"
#EVAL_CSV = "data/eval_set.csv"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LENGTH = 1024
target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']
columns_to_vectorize = ["prompt", "response_a", "response_b"]

train = pd.read_csv(TRAIN_CSV)
#eval_df = pd.read_csv(EVAL_CSV)

# Select the last 30,000 rows and reverse the order
#train = train.iloc[-11000:][::-1].reset_index(drop=True)
# Display the first few rows to verify
train = train[40000:]
train['label'] = train[target_columns].idxmax(axis=1) 
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['label'])
train = train[columns_to_vectorize + ['label']]
print(train['label'].value_counts())

print(len(train))

# Display the first few rows to verify the changes
print(train['label'].value_counts())

# Calculate the number of samples per label
# Determine the unique labels

# Display the first few rows to verify
print(train['label'].value_counts())

label
0    6095
1    5901
2    5481
Name: count, dtype: int64
17477
label
0    6095
1    5901
2    5481
Name: count, dtype: int64
label
0    6095
1    5901
2    5481
Name: count, dtype: int64


In [ ]:
'''# Step 1: Calculate the text length in the original DataFrame
train['text_length'] = train['prompt'].apply(len) + train['response_a'].apply(len) + train['response_b'].apply(len)

# Sort the DataFrame based on text length in descending order (longest texts first)
train_sorted = train.sort_values(by='text_length', ascending=False)

# Select the top 5% longest texts directly
subsample_size_20 = int(0.05 * len(df))
train_sorted = train_sorted.head(subsample_size_20)

merged_df = pd.concat([train_sorted, eval_df]).drop_duplicates().reset_index(drop=True)'''

In [4]:
tokenizer = AutoTokenizer.from_pretrained("internlm/internlm2-7b-reward", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
tokenizer.add_eos_token = True

# save tokenizer to load offline during inference
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/./tokenizer.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [5]:
def preprocess_chat(tokenizer, conversation):
    # Apply the chat template and encode the conversation
    conversation_str = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=False)
    # Encode without adding special tokens automatically
    input_ids = tokenizer.encode(conversation_str, return_tensors="pt", add_special_tokens=False,padding="max_length", truncation=True, max_length=MAX_LENGTH)

    # Check if the last token is not equal to 92527
    if input_ids[0, -1] != 92527:
        if input_ids.shape[1] >= MAX_LENGTH:
            # Remove the last word if max length is reached
            input_ids = input_ids[:, :-1]
        
        # Add the token 92527 at the end
        input_ids = torch.cat([input_ids, torch.tensor([[92527]], dtype=torch.long)], dim=1)

    # Ensure that the input_ids are exactly MAX_LENGTH tokens long
    if input_ids.shape[1] > MAX_LENGTH:
        input_ids = input_ids[:, :MAX_LENGTH]
    
    # Create attention mask
    attention_mask = torch.ones_like(input_ids, dtype=torch.bool)

    return {"input_ids": input_ids, "attention_mask": attention_mask}

def get_tokens(tokenizer, prompt, response_a, response_b):
    # Prepare the conversation with assistant_a and assistant_b roles
    chat = [
        {"role": "user", "content": prompt},
        {"role": "assistant_a", "content": response_a},
        {"role": "assistant_b", "content": response_b}
    ]
    
    # Tokenize the entire conversation
    tokens = preprocess_chat(tokenizer, chat)
    
    return tokens

def tokenize(example, tokenizer):
    prompt = example['prompt']
    response_a = example['response_a']
    response_b = example['response_b']
    
    # Get tokens for both assistant_a and assistant_b content
    tokens = get_tokens(tokenizer, prompt, response_a, response_b)
    
    # Extract input IDs and attention mask
    input_ids = tokens["input_ids"]
    attention_mask = tokens["attention_mask"]

    # If exceeding max length, truncate
    if input_ids.size(1) > MAX_LENGTH:
        input_ids = input_ids[:, :MAX_LENGTH - 1]
        attention_mask = attention_mask[:, :MAX_LENGTH - 1]

    # Assign label: 0 for Response A, 1 for Response B, 2 for Tie
    label = int(example['label'])  # Convert label to an integer based on your label encoding
    
    return {
        "input_ids": input_ids.squeeze(0).tolist(),  # Flatten to list
        "attention_mask": attention_mask.squeeze(0).tolist(),  # Flatten to list
        "labels": label
    }

def load_data(df, tokenizer):
    raw_datasets = Dataset.from_pandas(df)
    tokenized_datasets = raw_datasets.map(
        tokenize, 
        remove_columns=raw_datasets.column_names,
        fn_kwargs={'tokenizer': tokenizer}
    )
    return tokenized_datasets

In [6]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    print(labels)
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    loss = log_loss(y_true=labels, y_pred=probs)
    y_pred=preds.argmax(-1)
    print(y_pred)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {"acc": acc, "log_loss": loss}

n_splits = 100
fold_idx = 0
ds = load_data(train, tokenizer)
folds = [
    (
        [i for i in range(len(ds)) if i % n_splits != fold_idx],
        [i for i in range(len(ds)) if i % n_splits == fold_idx]
    ) 
    for fold_idx in range(n_splits)
]
train_idx, eval_idx = folds[fold_idx]

Map:   0%|          | 0/17477 [00:00<?, ? examples/s]

In [7]:
eval_df = train.iloc[eval_idx]

# Step 2: Save the filtered DataFrame to a CSV file
eval_df.to_csv('data/eval_set-3.csv', index=False)

In [8]:
ds[0]['input_ids'][-1]

92527

In [9]:
# Load your model configuration
config = AutoConfig.from_pretrained("quantized-model", num_labels=3,trust_remote_code=True)

In [10]:
# Load the model with ignore_mismatched_sizes=True
model = InternLM2ForSequenceClassification.from_pretrained(
    'quantized-model',
    config=config,
    trust_remote_code=True,
    device_map='auto',
    torch_dtype=torch.float16
)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Assuming `model` is your initialized model
total_params = count_parameters(model)

print(f"Total trainable parameters: {total_params:,}")


Total trainable parameters: 379,338,752


In [12]:
model

InternLM2ForSequenceClassification(
  (model): InternLM2Model(
    (tok_embeddings): Embedding(92544, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x InternLM2DecoderLayer(
        (attention): InternLM2Attention(
          (wqkv): Linear4bit(in_features=4096, out_features=6144, bias=False)
          (wo): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): InternLM2RotaryEmbedding()
        )
        (feed_forward): InternLM2MLP(
          (w1): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (w3): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (w2): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (attention_norm): InternLM2RMSNorm()
        (ffn_norm): InternLM2RMSNorm()
      )
    )
    (norm): InternLM2RMSNorm()
  )
  (v_head): Linear(in_features=4096, out_features=3, bias=False)
)

In [13]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias='none',
    inference_mode=False,
    task_type=TaskType.SEQ_CLS,
    target_modules=['wqkv', 'wo','w1','w2','w3'], 
)

In [14]:
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
print(model)
model.print_trainable_parameters()

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): InternLM2ForSequenceClassification(
      (model): InternLM2Model(
        (tok_embeddings): Embedding(92544, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-31): 32 x InternLM2DecoderLayer(
            (attention): InternLM2Attention(
              (wqkv): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=6144, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=6144, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector)

In [15]:
# Load the state_dict from the pytorch_model.bin file
state_dict = torch.load('output/H100-full_model_pytorch-2/pytorch_model.bin')

In [16]:
# Load the weights into the model, excluding the new target modules
missing_keys, unexpected_keys = model.load_state_dict(state_dict, strict=False)

# Print out any missing or unexpected keys for debugging
print(f"Missing keys: {missing_keys}")
print(f"Unexpected keys: {unexpected_keys}")

Missing keys: []
Unexpected keys: []


In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Assuming `model` is your initialized model
total_params = count_parameters(model)

print(f"Total trainable parameters: {total_params:,}")

Total trainable parameters: 37,748,736


In [18]:
# Unfreeze LoRA and v_head parameters
for name, param in model.named_parameters():
    if "lora_" in name or "v_head" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Optionally, you can print out which parameters are trainable
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Trainable parameter: {name}")

Trainable parameter: base_model.model.model.layers.0.attention.wqkv.lora_A.default.weight
Trainable parameter: base_model.model.model.layers.0.attention.wqkv.lora_B.default.weight
Trainable parameter: base_model.model.model.layers.0.attention.wo.lora_A.default.weight
Trainable parameter: base_model.model.model.layers.0.attention.wo.lora_B.default.weight
Trainable parameter: base_model.model.model.layers.0.feed_forward.w1.lora_A.default.weight
Trainable parameter: base_model.model.model.layers.0.feed_forward.w1.lora_B.default.weight
Trainable parameter: base_model.model.model.layers.0.feed_forward.w3.lora_A.default.weight
Trainable parameter: base_model.model.model.layers.0.feed_forward.w3.lora_B.default.weight
Trainable parameter: base_model.model.model.layers.0.feed_forward.w2.lora_A.default.weight
Trainable parameter: base_model.model.model.layers.0.feed_forward.w2.lora_B.default.weight
Trainable parameter: base_model.model.model.layers.1.attention.wqkv.lora_A.default.weight
Trainabl

In [19]:
model.print_trainable_parameters()

trainable params: 37,761,024 || all params: 7,396,409,344 || trainable%: 0.5105


In [21]:
args = TrainingArguments(
    output_dir='output-h100-3',
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="steps",
    save_steps=750,  # Save every 75 steps
    logging_strategy="steps",
    logging_steps=1,  # Log every 1 steps
    warmup_steps=150,  # Warmup steps
    optim="adamw_8bit",
    learning_rate=1e-6,  # Learning rate # 5e-6 # 7.5e-6 # 1e-6
    per_device_train_batch_size=8,  # Batch size of 8 sequences
    per_device_eval_batch_size=32, 
    gradient_accumulation_steps=2,  # Effectively 2 gradient accumulation
    max_grad_norm=0.4,  # Gradient clipping
    num_train_epochs=1,  # Number of epochs
    weight_decay=0.01,
    fp16=True,  # Mixed precision training
    metric_for_best_model="log_loss",
    greater_is_better=False,
    report_to="none",
)


/home/ubuntu/.local/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
trainer = Trainer(
    args=args,
    model=model,
    train_dataset=ds.select(train_idx),
    eval_dataset=ds.select(eval_idx),
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


In [23]:
print("## Starting to train ##")

## Starting to train ##


In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss,Acc,Log Loss,Runtime,Samples Per Second,Steps Per Second
0,0.959600,0.944280,0.411429,1.132267,26.977600,6.487000,0.222000


[1 0 2 1 0 1 2 2 1 1 0 2 2 2 2 2 2 2 2 2 1 0 1 1 0 0 1 1 0 2 1 2 2 0 1 0 1
 0 2 0 0 0 0 2 1 0 2 1 1 0 0 2 0 2 0 0 1 0 2 1 1 2 1 1 1 0 0 2 1 1 0 2 1 2
 2 2 0 0 0 0 0 0 1 1 0 2 0 1 1 1 2 0 0 1 1 0 1 1 2 2 1 2 1 1 1 0 1 1 0 1 0
 1 0 2 0 2 2 2 0 1 1 2 1 0 2 2 1 0 0 0 1 0 1 0 0 1 0 2 1 1 2 1 1 2 0 0 2 1
 1 2 0 1 1 2 2 0 2 1 2 1 0 0 1 0 2 1 0 2 0 0 1 0 0 2 0]
[2 1 2 0 0 0 0 0 0 1 1 2 1 0 2 2 2 1 0 0 1 2 2 1 0 2 2 0 2 1 2 0 2 0 1 1 1
 0 2 2 0 0 2 1 1 0 2 2 1 0 0 1 0 2 1 1 2 2 0 0 1 2 1 2 2 2 1 2 2 0 1 2 1 2
 2 2 0 0 1 0 1 2 0 1 1 2 1 0 2 1 1 0 1 2 1 2 2 2 0 1 1 0 0 0 1 1 0 2 0 1 2
 1 2 1 0 1 0 0 2 2 1 2 0 1 2 0 0 0 2 1 0 0 0 0 0 0 2 1 1 1 0 2 2 1 0 2 1 0
 0 2 0 0 1 2 0 1 2 1 0 1 0 2 1 2 1 2 1 1 2 0 1 0 2 1 0]


TrainOutput(global_step=1081, training_loss=0.8671687980915638, metrics={'train_runtime': 9472.3499, 'train_samples_per_second': 1.827, 'train_steps_per_second': 0.114, 'total_flos': 7.457100004117709e+17, 'train_loss': 0.8671687980915638, 'epoch': 0.9995376791493297})

In [25]:
save_path = 'output/H100-full_model_pytorch-3'

# Ensure the directory exists
os.makedirs(save_path, exist_ok=True)

# Save the full model, including the v_head layer
torch.save(model.state_dict(), os.path.join(save_path, 'pytorch_model.bin'))

# Save the configuration as well
model.config.save_pretrained(save_path)

In [26]:
model = model.merge_and_unload()
model.save_pretrained('H100-merged-3')

/home/ubuntu/.local/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
